In [1]:
import pandas as pd
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor

In [2]:
stats = pd.read_csv("player_mvp_stats.csv")
del stats["Unnamed: 0"]
stats_all_players = stats.fillna(0)
mvp_players = stats_all_players[(stats_all_players["FG%"] > 0.384) & (stats_all_players["MP"] > 25.6) & (stats_all_players["FT%"] > 0.422) & (stats_all_players["PTS"] > 4.7) & (stats_all_players["VORP"] > 1.1)& (stats_all_players["W/L%"] > 0.22)]
mvp_players = stats_all_players

In [3]:
stats_2024 = pd.read_csv("2024_stats.csv")
del stats_2024["Unnamed: 0"]

In [4]:
stats_2024 = stats_2024.fillna(0)
stats_2024 = stats_2024[(stats_2024["FG%"] > 0.384) & (stats_2024["MP"] > 25.6) & (stats_2024["FT%"] > 0.422) & (stats_2024["PTS"] > 4.7) & (stats_2024["VORP"] > 1.1)& (stats_2024["W/L%"] > 0.22)]
condition = (stats_2024['W'] + stats_2024['L']) - stats_2024['G'] < 17
stats_2024 = stats_2024[condition]

In [5]:
predictors = ['Age', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P',
       '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB',
       'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'Year', 'W', 'L', 'W/L%', 'GB', 'PS/G',
       'PA/G', 'SRS','PER','TS%', '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%',
       'TOV%', 'USG%', 'OWS', 'DWS', 'WS', 'WS/48', 'OBPM', 'DBPM', 'BPM',
       'VORP']

#Predicting on 2024 players stats. Training model on data from previous years (1990 - 2023)
train = mvp_players
test = stats_2024

X_train = train[predictors]
y_train = train["Share"]

X_test = test[predictors]
# y_test = test["Share"]

model = XGBRegressor(learning_rate = 0.05, max_depth = 7, subsample = 0.5)

model.fit(X_train, y_train)
predicted_winners = model.predict(X_test)

predicted_winners = pd.DataFrame(predicted_winners, columns=["Predictions"], index = test.index)
combinations_XGB = pd.concat([test, predicted_winners], axis = 1)
combinations_XGB = combinations_XGB.sort_values("Predictions", ascending = False)
# combinations = combinations.sort_values("Share", ascending = False)
# combinations["Rank"] = list(range(1, combinations.shape[0] + 1))
# combinations = combinations.sort_values("Predictions", ascending = False)
# combinations["Predicted_Rank"] = list(range(1, combinations.shape[0] + 1))

In [6]:
combinations_XGB.head(10)

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,VORP,Team,W,L,W/L%,GB,PS/G,PA/G,SRS,Predictions
69,Nikola Jokić,C,28,DEN,78,78,34.7,10.4,17.9,0.582,...,10.5,Denver Nuggets,56,25,0.691,0.0,114.7,109.6,5.18,0.984569
125,Shai Gilgeous-Alexander,PG,25,OKC,74,74,34.3,10.7,19.9,0.536,...,7.0,Oklahoma City Thunder,56,25,0.691,0.0,119.9,113.0,6.83,0.728048
32,Luka Dončić,PG,24,DAL,70,70,37.5,11.5,23.6,0.487,...,8.0,Dallas Mavericks,50,31,0.617,0.0,118.2,115.4,2.82,0.552732
7,Giannis Antetokounmpo,PF,29,MIL,73,73,35.2,11.5,18.8,0.611,...,7.2,Milwaukee Bucks,49,32,0.605,0.0,119.4,116.4,2.75,0.538559
225,Domantas Sabonis,C,27,SAC,81,81,35.8,7.7,13.1,0.593,...,6.1,Sacramento Kings,45,36,0.556,6.0,116.5,115.2,1.94,0.128639
187,Jalen Brunson,PG,27,NYK,76,76,35.3,10.2,21.3,0.480,...,5.4,New York Knicks,49,32,0.605,14.0,112.7,108.1,4.43,0.094417
107,Tyrese Haliburton,PG,23,IND,68,67,32.3,7.3,15.3,0.477,...,5.0,Indiana Pacers,46,35,0.568,3.0,122.9,120.3,2.33,0.073801
334,Anthony Edwards,SG,22,MIN,78,77,35.1,9.2,19.9,0.461,...,3.7,Minnesota Timberwolves,56,25,0.691,0.0,113.1,106.3,6.65,0.059781
327,LeBron James,PF,39,LAL,70,70,35.2,9.6,17.8,0.540,...,5.1,Los Angeles Lakers,46,35,0.568,5.0,118.0,117.6,0.82,0.055519
168,Jayson Tatum,PF,25,BOS,74,74,35.7,9.1,19.3,0.471,...,4.7,Boston Celtics,63,18,0.778,0.0,120.4,109.1,10.87,0.052776


In [7]:
combinations_XGB.head(5).to_csv("XGB_predictions.csv")

In [8]:
model = RandomForestRegressor(bootstrap=True, max_depth=40, max_features = 'sqrt', min_samples_leaf=2, min_samples_split=2, n_estimators = 200)

model.fit(X_train, y_train)
predicted_winners = model.predict(X_test)

predicted_winners = pd.DataFrame(predicted_winners, columns=["Predictions"], index = test.index)
combinations_RF = pd.concat([test, predicted_winners], axis = 1)
combinations_RF = combinations_RF.sort_values("Predictions", ascending = False)

In [9]:
combinations_RF.head(10)

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,VORP,Team,W,L,W/L%,GB,PS/G,PA/G,SRS,Predictions
69,Nikola Jokić,C,28,DEN,78,78,34.7,10.4,17.9,0.582,...,10.5,Denver Nuggets,56,25,0.691,0.0,114.7,109.6,5.18,0.804275
125,Shai Gilgeous-Alexander,PG,25,OKC,74,74,34.3,10.7,19.9,0.536,...,7.0,Oklahoma City Thunder,56,25,0.691,0.0,119.9,113.0,6.83,0.623015
7,Giannis Antetokounmpo,PF,29,MIL,73,73,35.2,11.5,18.8,0.611,...,7.2,Milwaukee Bucks,49,32,0.605,0.0,119.4,116.4,2.75,0.510719
32,Luka Dončić,PG,24,DAL,70,70,37.5,11.5,23.6,0.487,...,8.0,Dallas Mavericks,50,31,0.617,0.0,118.2,115.4,2.82,0.441167
168,Jayson Tatum,PF,25,BOS,74,74,35.7,9.1,19.3,0.471,...,4.7,Boston Celtics,63,18,0.778,0.0,120.4,109.1,10.87,0.119699
187,Jalen Brunson,PG,27,NYK,76,76,35.3,10.2,21.3,0.480,...,5.4,New York Knicks,49,32,0.605,14.0,112.7,108.1,4.43,0.088032
315,Anthony Davis,C,30,LAL,75,75,35.6,9.3,16.9,0.553,...,4.7,Los Angeles Lakers,46,35,0.568,5.0,118.0,117.6,0.82,0.081515
225,Domantas Sabonis,C,27,SAC,81,81,35.8,7.7,13.1,0.593,...,6.1,Sacramento Kings,45,36,0.556,6.0,116.5,115.2,1.94,0.073293
107,Tyrese Haliburton,PG,23,IND,68,67,32.3,7.3,15.3,0.477,...,5.0,Indiana Pacers,46,35,0.568,3.0,122.9,120.3,2.33,0.046683
327,LeBron James,PF,39,LAL,70,70,35.2,9.6,17.8,0.540,...,5.1,Los Angeles Lakers,46,35,0.568,5.0,118.0,117.6,0.82,0.041503


In [10]:
combinations_RF.head(5).to_csv("RF_predictions.csv")